# pysat demo using pysatMissions orbit simulations and associate methods

In [ ]:
import datetime as dt

In [ ]:
import pysat
import pysatMissions

In [ ]:
# Set data directory if user hasn't already set one
if len(pysat.params['data_dirs']) == 0:
    # Set a directory for pysat to use for data
    pysat.params['data_dirs'] = '~/shared/pysatDemo'
else:
    print('pysat directory has been set previously. Leaving unchanged.')

# Register the data plug-ins in pysatMissions. Only once per installation.
pysat.utils.registry.register_by_module(pysatMissions.instruments)

In [ ]:
# Define on-the-fly orbit breakdown input. Note that `mlt` variable will be added by custom function.
orbit_info = {'kind': 'lt', 'index': 'mlt', 'period': dt.timedelta(minutes=95)}

In [ ]:
# Instantiate satellite propagator. Note that `inclination` and `alt_periapsis` are keywords defined by 
# `missions_sgp4` module, not pysat itself.
inst = pysat.Instrument('missions', 'sgp4', orbit_info=orbit_info, inclination=10, alt_periapsis=500.)

In [ ]:
# Get information on SGP4 support as well as defined keyword arguments.
help(inst.inst_module)

In [ ]:
# Add additional information, like location in magnetic coordinates, including `mlt`.
inst.custom_attach(pysatMissions.methods.magcoord.add_quasi_dipole_coordinates, 
                   kwargs={'glong_label': 'geod_longitude', 'glat_label': 'geod_latitude',
                           'alt_label': 'geod_altitude' })

In [ ]:
# Load a day of data and plot raw `mlt`. pysat appplies all custom functions during load process.
inst.load(2019, 2, use_header=True)
inst['mlt'].plot(title='Simulated Spacecraft Day', ylabel='MLT (hours)')

In [ ]:
# Check out data before orbit breakdown. Full day of data.
inst.data

In [ ]:
# Iterate orbit-by-orbit. Plot the first orbit then stop. To create this plot pysat also loads/simulates
# day previous to ensure that the first orbit is complete. 
for orbit_inst in inst.orbits:
    orbit_inst.data.plot(y='latitude', x='mlt', title='Single Orbit', ylabel='Latitude (degrees)')
    break

In [ ]:
# Check out data after first orbit breakdown. Single orbit of data. Notice the first samples come from 2019, 1.
# pysat does its best to form complete orbits when possible, with a minimum of data loading. 
inst.data

In [ ]:
# The for loop above first takes some time to calculate as it is loading/simulating data.
# Now that is complete, moving forward in orbits will be fast until we get close to the end of the day.
# pysat will then load the next day, with the communserate simulation time.
# Move forward with orbits.next(), backward with orbits.prev(), and select particular orbit with orbits[]

inst.orbits.next()
# inst.orbits.prev()
# inst.orbits[5]

# Title string
date_str = '%x %X'
title = ''.join(['Orbit: ', repr(inst.orbits.current), '   ', inst.index[0].strftime(date_str), 
                 ' - ', inst.index[-1].strftime(date_str)])

# Make plot
inst['mlt'].plot(title=title, ylabel='MLT (hours)')